In [1]:
import os
import numpy as np
import torch

In [10]:
from Config import parse_arguments
from Models.model_utils import FEDMD_digest_revisit, save_checkpoints, select_model, get_logits, create_model_architectures_noblip
from Common.dataset_fedmd import FEDMD_load_dataloaders_noblip, FEDMD_load_datasets_noblip

In [3]:
args = parse_arguments()

In [4]:
args

Namespace(sample_fraction=1.0, min_num_clients=2, KD=False, algo_type='FEDMD_homogen', available_architectures_noblip=['NetS', 'DeepNN_Ram', 'DeepNN_Hanu', 'DeepNN_Lax'], available_architectures_blip=['MLP1', 'MLP2', 'MLP3', 'MLP4'], name='cifar100', partitioning='dirichlet', num_clients=10, num_rounds=20, BLIP=False, proximal_mu=0.02, scheduler_=False, val_ratio=0.2, seed=42, alpha=1.0, labels_per_client=2, similarity=0.5, batch_size=32, batch_size_ratio=0.01, server_epochs=5, num_cpus=1, num_gpus=1, device='cuda')

In [5]:
def fit_config(server_round: int):
    """Return training configuration dict for each round."""

    config = {
        "server_round": server_round,
        "epochs": 2,
        "proximal_mu":args.proximal_mu,
        "client_kd_alpha": 0.3,
        "client_kd_temperature": 3,
        "server_kd_alpha": 0.7,
        "server_kd_temperature": 7,
        "server_lr": 0.001 
            }
    return config

In [6]:
    
def trasfer_learning_init(args, model_architectures, serverloader, trainloaders):
    """
    Initialize the model for transfer learning.
    This function trains the model on the server data and then initializes client models.
    """

    client_logits = {}
    config = fit_config(server_round=0)
    num_classes=100

    for i, trainloader in enumerate(trainloaders):
        print(f"\n Transfer learning on client {i + 1}\n")

        net = select_model(model_architectures[i], args.device, args.BLIP, num_classes=num_classes)
        optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
        # Training the model on public data
        FEDMD_digest_revisit(serverloader, net, optimizer, args.device, config, aggregated_logits=None, mode="revisit")
        # Training the model on private data
        FEDMD_digest_revisit(trainloader, net, optimizer, args.device, config, aggregated_logits=None, mode="revisit")
        # logit Communication for round 1
        client_logits[i] = get_logits(serverloader, net, args.device)
        # Save the model and optimizer state for each client
        checkpoint_dir = f"client_checkpoints/{args.algo_type}/{args.alpha}/client_{i}"
        os.makedirs(checkpoint_dir, exist_ok=True)
        save_checkpoints(net, optimizer, 0, checkpoint_dir)

    logits = np.vstack([client_logits[i] for i in sorted(client_logits.keys())])

    return logits

In [7]:
trainloaders, valloaders, testloader, server_loader = FEDMD_load_dataloaders_noblip(args)
results_save_path = f"./results/NO_BLIP/{args.algo_type}/{args.alpha}"
model_architectures = create_model_architectures_noblip(args.available_architectures_noblip, args.algo_type, args.num_clients)

In [ ]:
0,2,20,63,71,82

In [14]:
from torch.utils.data import DataLoader, random_split, TensorDataset
from torch.utils.data import ConcatDataset, Dataset, Subset
from torchvision.datasets import CIFAR10, MNIST, CIFAR100
import torchvision.transforms as transforms

def load_CIFAR_data(data_type="CIFAR10", label_mode="fine", standardized=False, verbose=False):    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]) if standardized else transforms.ToTensor()

    if data_type == "CIFAR10":
        train_set = CIFAR10(root='./data/data_cifar10', train=True, download=True, transform=transform)
        test_set = CIFAR10(root='./data/data_cifar10', train=False, download=True, transform=transform)
        X_train, y_train = train_set.data, np.array(train_set.targets)
        X_test, y_test = test_set.data, np.array(test_set.targets)
    elif data_type == "CIFAR100":
        train_set = CIFAR100(root='./data/data_cifar100', train=True, download=True, transform=transform)
        test_set = CIFAR100(root='./data/data_cifar100', train=False, download=True, transform=transform)
        X_train, y_train = train_set.data, np.array(train_set.targets)  # Fine labels by default
        X_test, y_test = test_set.data, np.array(test_set.targets)
        
        if label_mode == "coarse":
            # Use coarse labels instead of fine labels
            y_train = np.array(train_set.coarse_targets)
            y_test = np.array(test_set.coarse_targets)
    else:
        print("Unknown Data type. Stopped!")
        return None

    if verbose:
        print("X_train shape:", X_train.shape)
        print("X_test shape:", X_test.shape)
        print("y_train shape:", y_train.shape)
        print("y_test shape:", y_test.shape)
        if data_type == "CIFAR100":
            print(f"Using {label_mode} labels")
            print("Label distribution:", np.unique(y_train, return_counts=True))
    
    return X_train, y_train, X_test, y_test

In [25]:
X_train, y_train, X_test, y_test = load_CIFAR_data(data_type="CIFAR100", label_mode="coarse", standardized=False, verbose=True)

AttributeError: 'CIFAR100' object has no attribute 'coarse_targets'

In [17]:
X_train.shape

(50000, 32, 32, 3)

In [24]:
set(y_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [20]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]) 
train_set = CIFAR100(root='./data/data_cifar100', train=True, download=True, transform=transform)

In [22]:
np.array(train_set.coarse_targets)

AttributeError: 'CIFAR100' object has no attribute 'coarse_targets'